## Neural Networks From Scratch

Here we will build a simple feed forward artificial Neural network from scratch

First step by step and then functionise it.

In [6]:
#first of all we will need some packages
import numpy as np
import pandas as pd
import math
import random

In [4]:
#lets create a small numpy array of input data, this is the same as our deforestation data
Input = np.array([10,1,4])

In [41]:
#take a look at it
Input

array([10,  1,  4])

In [42]:
#Ok for dimensional reasons we want to transpose our Input

In [44]:
Input = Input.reshape((Input.shape[0],1))

In [45]:
Input

array([[10],
       [ 1],
       [ 4]])

From the lecture we had a simple structure:

3 Input : 4 Hidden Layer : 1 Output

In [126]:
#So lets generate an array that is the weights our hidden layer of size 4 
# the weights need to be generated randomly and we will generate them between -4 and +4 for ease
mean = 0
stddev = 3
Hidden = np.array([random.gauss(mean,stddev) for _ in range(12)])

In [127]:
Hidden

array([-0.19025482,  0.1001729 ,  2.97870522,  6.34286942,  4.0150106 ,
       -1.91301903, -2.36477718,  0.12299034,  0.02835017, -1.44166167,
       -6.54371261, -1.45838569])

In [128]:
#We want to make sure that we have the correct dimensions here
Hidden = Hidden.reshape(len(Input),4)

In [129]:
Hidden

array([[-0.19025482,  0.1001729 ,  2.97870522,  6.34286942],
       [ 4.0150106 , -1.91301903, -2.36477718,  0.12299034],
       [ 0.02835017, -1.44166167, -6.54371261, -1.45838569]])

In [130]:
#Lets also generate some random biases
BiasHidden = np.array([random.gauss(mean,stddev) for _ in range(4)])

In [131]:
BiasHidden

array([-0.54127319,  3.04424858, -4.19162685, -2.37510582])

In [132]:
#Ok so we have our input and our hidden layer now we want to work out the first layer
#which means we need to do our first multiplication

Layer1 = np.dot(Input.T,Hidden)

In [133]:
#And here are our outputs for layer 1 after multiplication
Layer1

array([[ 2.22586306, -6.67793669,  1.24742459, 57.71814174]])

In [134]:
#Remember to add the biases
Layer1 = Layer1+BiasHidden

In [135]:
Layer1

array([[ 1.68458987, -3.63368811, -2.94420226, 55.34303592]])

In [98]:
#Now we have to apply our activation functions, we covered a few activation functions in the lecture and specifically for the hidden layer
#we looked at ReLu and tanh so we will define them here as functions

#ReLU is simple as it just turns any negative values to 0
def ReLU(x):
    return np.maximum(x,0)

In [136]:
ReLU(Layer1)

array([[ 1.68458987,  0.        ,  0.        , 55.34303592]])

In [109]:
#tanh is a bit more complicated taking the form e^x - e^-x / e^x + e^-x
def tanh(x):
    ex = np.exp(x)
    enx = np.exp(-x)
    return (ex - enx) / (ex + enx)

In [137]:
tanh(Layer1)

array([[ 0.93345463, -0.99860509, -0.99447253,  1.        ]])

In [138]:
#Tanh is actually already inside numpy
np.tanh(Layer1)

array([[ 0.93345463, -0.99860509, -0.99447253,  1.        ]])

In [139]:
#lets use tanh as our activation function 
Layer1out = tanh(Layer1)

In [140]:
Layer1out

array([[ 0.93345463, -0.99860509, -0.99447253,  1.        ]])

In [141]:
#next we will need the weights of our output layer we are trying to collapse 4 into 1 so we need 4 weights
OutWeights = np.array([random.gauss(mean,stddev) for _ in range(4)])

In [142]:
OutWeights

array([ 0.02096114,  4.34008912, -2.85926929,  3.17636012])

In [205]:
Out = np.dot(Layer1out,OutWeights)

In [208]:
Out

array([1.70535606])

In [206]:
#We need a bias for the final one
BiasOut = np.array([random.gauss(mean,stddev) for _ in range(1)])

In [207]:
BiasOut

array([-5.18150484])

In [209]:
Out += BiasOut

In [122]:
#Now we need our final activation function the sigmoid function
def Sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [210]:
OutPrediction = Sigmoid(Out)

In [211]:
OutPrediction

array([0.02999854])

In [212]:
#Ok we have done a complete forward pass through our neural network and the model is predicting this area will be deforested with a probability
#as this area was deforested it should be 1 so lets calculate the error, we will make a function that does this
def LogLoss(x):
    return -np.log(x)

In [213]:
Error = LogLoss(OutPrediction)

In [214]:
Error

array([3.50660648])

In [153]:
#Now we need to begin our back propagation with this error the first step will need the derivative of the sigmoid function
# the dsigmoid = sigmoid * 1 - Sigmoid
def SigmoidDerivative(x):
    return (1 / (1 + np.exp(-x))) * (1 - (1 + np.exp(-x)))

In [154]:
#we also need our log loss derivative which is just -1/-logloss
def DLogLoss(x):
    return 1/(-x)

In [215]:
#our error composite then becomes DLogLoss * Sigmoid Derivative
Errorcomp = DLogLoss(Error) * SigmoidDerivative(OutPrediction)

In [216]:
Errorcomp

array([-0.39254088])

In [254]:
#now we back propagate this through the weights
grads1 = np.dot(Errorcomp, Layer1out)

In [255]:
grads1

array([-0.3664191 ,  0.39199332,  0.39037112, -0.39254088])

In [256]:
#Now to make the change in bias which is the sum of the changes in weights
bias1 = sum(grads1)

In [258]:
bias1

0.02340445992664336

In [168]:
#next we need to backcast the error into the final layer of weights
#for this we will need the differential of the tanh activation function that we used
def DTanh(x):
    return 1 - np.tanh(x)**2

In [221]:
DLayer1 = DTanh(Layer1out)

In [222]:
DLayer1

array([[0.46388243, 0.42086727, 0.42351974, 0.41997434]])

In [223]:
#now we multiple this by our change in the last layer or grads 1
Dstep1 = DLayer1 * grads1

In [224]:
Dstep1

array([[-0.30675172,  0.60031292,  0.48947013, -9.12369228]])

In [225]:
#we have this step which is the error and the differential of layer 1 together we now need to multiply this by the input to get our weights
grads2 = np.dot(Input,Dstep1)

In [226]:
grads2

array([[ -3.06751716,   6.00312922,   4.89470129, -91.23692285],
       [ -0.30675172,   0.60031292,   0.48947013,  -9.12369228],
       [ -1.22700686,   2.40125169,   1.95788052, -36.49476914]])

In [311]:
bias2 = sum(sum(grads2))

In [312]:
bias2

-125.10991423501088

In [230]:
#now we have our changes in weights and changes in biases we just need to add these to our weights and biases
Hidden += grads2
OutWeights += grads1
BiasHidden += bias2
BiasOut += bias1

And that is that, we have updated all the weights and biases in one full pass if we run it through again it would hopefully perform better

Now we should turn it into a nice function.

In [282]:
#lets start with a function that creates the network
def InitialiseNetwork(Inputsize,HiddenSize,OutSize):
    mean = 0
    stddev = 3
    Hidden = np.array([random.gauss(mean,stddev) for _ in range(Inputsize * HiddenSize)])
    Hidden = Hidden.reshape(Inputsize,HiddenSize)
    BiasHidden = np.array([random.gauss(mean,stddev) for _ in range(HiddenSize)])
    OutWeights = np.array([random.gauss(mean,stddev) for _ in range(HiddenSize * OutSize)])
    OutWeights = OutWeights.reshape(OutSize,HiddenSize)
    BiasOut = np.array([random.gauss(mean,stddev) for _ in range(OutSize)])
    return dict(Hidden = Hidden, BiasHidden = BiasHidden,OutWeights = OutWeights,BiasOut = BiasOut )
    
                          

In [283]:
Net1 = InitialiseNetwork(3,4,1)

In [284]:
Net1

{'Hidden': array([[-2.54669762,  3.49544684,  2.97664393,  2.32692595],
        [ 3.97546497,  2.91907559, -2.60520249, -1.2279104 ],
        [-3.19311754,  0.84074965, -2.95835329, -2.27202845]]),
 'BiasHidden': array([-0.86205824,  3.14574142, -4.05423661, -2.83631578]),
 'OutWeights': array([[-1.3685571 ,  4.47570245,  0.1485243 , -0.07813973]]),
 'BiasOut': array([1.86184289])}

In [287]:
def ForwardPass(Input,Net):
    Forward1 = tanh((np.dot(Input.T,Net['Hidden'])) + Net['BiasHidden'])
    Forward2 = Sigmoid(np.dot(Forward1,Net['OutWeights'].T)+Net['BiasOut'])
    return dict(Forward1 = Forward1, Forward2 = Forward2)
    

In [288]:
FW1 = ForwardPass(Input,Net1)

In [289]:
FW1

{'Forward1': array([[-1.,  1.,  1.,  1.]]), 'Forward2': array([[0.99958069]])}

In [303]:
def BackwardsPass(Input,Forward,Net):
    Error = LogLoss(Forward['Forward2'])
    Errorcomp = DLogLoss(Forward['Forward2']) * SigmoidDerivative(Forward['Forward2'])
    grads1 = Errorcomp * Forward['Forward1']
    bias1 = sum(sum(grads1))
    grads2 = np.dot(Input,(grads1 * DTanh(Forward['Forward1'])))
    bias2 = sum(sum(grads2))
    Net['Hidden'] += grads2
    Net['BiasHidden'] += bias2
    Net['OutWeights'] += grads1
    Net['BiasOut'] += bias1
    return Net, Error

In [305]:
Net2,Err = BackwardsPass(Input,FW1,Net1)

In [306]:
Net2

{'Hidden': array([[-21554.17224042,  21555.12098964,  21554.60219037,
          21553.95250553],
        [ -2151.18708931,   2158.08162987,   2152.55735215,
           2153.93464756],
        [ -8623.84333466,   8621.49096677,   8617.69186528,
           8618.37820339]]),
 'BiasHidden': array([-8082.72162163, 40412.44364933, 40405.24367539, 40406.46163351]),
 'OutWeights': array([[-5133.02147189,  5136.12861723,  5131.80143743,  5131.57475832]]),
 'BiasOut': array([2567.68829568])}

In [308]:
Err

array([[0.0004194]])

There we go if we run this over and over on the same data it will improve

Try than now using a for loop:

Next thing to do is to make the neural network a class. All the functions are there for you all you need to do is make a class that works with any size:

## Task

As a final task for this practical we will build a neural network to classify some real world data. You should have two data sets called ForestClass either training or testing. These are real world datasets extracted from satelite images. The class variable is the type of forest that the data is from and the other data are all bandwidth measurements from band 1:9 of satelite images. Build a neural network that will predict the type of forest. For an easy version just try and classify between the two most common types or if you want more of a challenge you can try and classify between all of them, for which you will need the softmax function:

In [1]:
#softmax is very much like the sigmoid function but for multiple classes
def softmax(x):
   #the softmax is defined as e^x-max(x) / sum(e^x-max(x))
    e_x = np.exp(x - np.max(x))
    #it will return an array that is the softmax of an array the highest value in the softmax is your predicted class
    return e_x / e_x.sum(axis = 0)

In [2]:
#the softmax derivative is incredibly complicated as it uses a masking jacobian matrix but you will also need it
def softmax_dif(s):
    #if this isn't working try and transpose your softmax output
    jacobian_m = np.diag(s)

    for i in range(len(jacobian_m)):
        for j in range(len(jacobian_m)):
            if i == j:
                jacobian_m[i][j] = s[i] * (1-s[i])
            else: 
                jacobian_m[i][j] = -s[i]*s[j]
    return jacobian_m

Good Luck